<a href="https://colab.research.google.com/github/pbaitor/i18n-sentiment/blob/main/notebook/Tweet_affect_analysis_with_individual_classifiers_(dec_2020).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

Install facebook LASER:

In [ ]:
!pip install laserembeddings

!python -m laserembeddings download-models

     |████████████████████████████████| 51kB 3.4MB/s 
     |████████████████████████████████| 860kB 10.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=884000 sha256=413fba0e0a142f7c165c046562a114d6d917f0d326c9ec7ebef2110aa98ac442
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses

✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


Install BM-MLL loss function:

In [ ]:
!pip install bpmll

Install NTLK utils

In [ ]:
from nltk import download as nltkdownload
nltkdownload('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Load datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np

X_dev_raw = np.genfromtxt('/content/drive/MyDrive/Machine Learning/Datasets/SemEval-2018 Affect in Tweets E-c/2018-E-c-En-dev.txt', skip_header=1, usecols=(1), dtype=str, delimiter="\t", comments="#@")
Y_dev = np.genfromtxt('/content/drive/MyDrive/Machine Learning/Datasets/SemEval-2018 Affect in Tweets E-c/2018-E-c-En-dev.txt', skip_header=1, usecols=list(range(2,13)), delimiter="\t", comments="#@")
# numpy.gentext assumes "#" as the start of a comment on text, the tweets dataset have a lot of # symbols

X_dev = np.array([s.replace('#', '') for s in X_dev_raw]) # remove all the # 

print(X_dev[0])
print(Y_dev[0])
print(X_dev.shape, "\n")

X_train_raw = np.genfromtxt('/content/drive/MyDrive/Machine Learning/Datasets/SemEval-2018 Affect in Tweets E-c/2018-E-c-En-train.txt', skip_header=1, usecols=(1), dtype=str, delimiter="\t", comments="#@")
Y_train = np.genfromtxt('/content/drive/MyDrive/Machine Learning/Datasets/SemEval-2018 Affect in Tweets E-c/2018-E-c-En-train.txt', skip_header=1, usecols=list(range(2,13)), delimiter="\t", comments="#@")
# numpy.gentext assumes "#" as the start of a comment on text, the tweets dataset have a lot of # symbols

X_train = np.array([s.replace('#', '') for s in X_train_raw]) # remove all the # 

print(X_train[0])
print(Y_train[0])
print(Y_train.shape, "\n")

X_test_raw = np.genfromtxt('/content/drive/MyDrive/Machine Learning/Datasets/SemEval-2018 Affect in Tweets E-c/2018-E-c-En-test-gold.txt', skip_header=1, usecols=(1), dtype=str, delimiter="\t", comments="#@")
Y_test = np.genfromtxt('/content/drive/MyDrive/Machine Learning/Datasets/SemEval-2018 Affect in Tweets E-c/2018-E-c-En-test-gold.txt', skip_header=1, usecols=list(range(2,13)), delimiter="\t", comments="#@")
# numpy.gentext assumes "#" as the start of a comment on text, the tweets dataset have a lot of # symbols

X_test = np.array([s.replace('#', '') for s in X_test_raw]) # remove all the # 

print(X_test[0])
print(Y_test[0])
print(Y_test.shape, "\n")


@RanaAyyub @rajnathsingh Oh, hidden revenge and anger...I rememberthe time,she rebutted you.
[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
(886,) 

“Worry is a down payment on a problem you may never have'.  Joyce Meyer.  motivation leadership worry
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
(6838, 11) 

@Adnan__786__ @AsYouNotWish Dont worry Indian army is on its ways to dispatch all Terrorists to Hell
[1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1.]
(3259, 11) 



In [ ]:
count = 0
for element in Y_train:
  if np.sum(element) == 0:
    count +=1
print(count)

204


In [ ]:
Y_train_clean = np.array([item for item in Y_train if np.sum(item)>0])
print("Cleaned Y_train from ", len(Y_train), " to ", len(Y_train_clean))

X_train_clean = np.array([X_train[i] for i, item in enumerate(Y_train) if np.sum(item)>0])
print("Cleaned X_train from ", len(X_train), " to ", len(X_train_clean))

Y_dev_clean = np.array([item for item in Y_dev if np.sum(item)>0])
print("Cleaned Y_dev from ", len(Y_dev), " to ", len(Y_dev_clean))

X_dev_clean = np.array([X_dev[i] for i, item in enumerate(Y_dev) if np.sum(item)>0])
print("Cleaned X_dev from ", len(X_dev), " to ", len(X_dev_clean))


Cleaned Y_train from  6838  to  6634
Cleaned X_train from  6838  to  6634
Cleaned Y_dev from  886  to  872
Cleaned X_dev from  886  to  872


In [ ]:
accum = sum(Y_train_clean)
print(np.around(accum/len(Y_train_clean),decimals=2))

[0.38 0.15 0.39 0.19 0.37 0.11 0.3  0.12 0.3  0.05 0.05]


In [ ]:
# Import package
import pandas as pd

filename='/content/drive/MyDrive/Machine Learning/Datasets/SemEval-2018 Affect in Tweets E-c/2018-E-c-En-test-gold.txt'
# Build DataFrame of tweet texts and languages. Use pd.DataFrame() to construct a DataFrame of tweet texts and languages; to do so, the first argument should be tweets_data, a list of dictionaries. The second argument to pd.DataFrame() is a list of the keys you wish to have as columns
df = pd.read_csv(filename, sep='\t', lineterminator='\r')

# Print head of DataFrame
print(df.head())
print(list(df))
print(df['Tweet'][0])

                ID  ... trust
0  \n2018-En-01559  ...   1.0
1  \n2018-En-03739  ...   0.0
2  \n2018-En-00385  ...   0.0
3  \n2018-En-03001  ...   0.0
4  \n2018-En-01988  ...   0.0

[5 rows x 13 columns]
['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']
@Adnan__786__ @AsYouNotWish Dont worry Indian army is on its ways to dispatch all Terrorists to Hell


# Generate sentence embeddings

In [ ]:
from laserembeddings import Laser

laser = Laser()

X_train_embeddings = laser.embed_sentences(X_train_clean,lang='en')
X_dev_embeddings = laser.embed_sentences(X_dev_clean,lang='en')
X_test_embeddings = laser.embed_sentences(X_test,lang='en')

In [ ]:
print(X_train_embeddings.shape)
print(X_dev_embeddings.shape)
print(X_test_embeddings.shape)

(6634, 1024)
(872, 1024)
(3259, 1024)


# Model the classifiers

In [ ]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.4.0
Running on TPU  ['10.90.93.42:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.90.93.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.90.93.42:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import BinaryCrossentropy

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

n = X_train_embeddings.shape[0]
dim_no = X_train_embeddings.shape[1]
class_no = Y_train_clean.shape[1]


# baseline model
def create_baseline(number_of_classes=class_no):
  # create model
  model = Sequential()
  model.add(Dense(128, input_dim=dim_no, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(number_of_classes, activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

labels = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

# evaluate model with stratified datasets
stats = []
for i in range(11):
  ordinal = i
  X_input = X_train_embeddings
  Y_output = Y_train_clean[:,ordinal]

  estimator = KerasClassifier(build_fn=create_baseline, number_of_classes=1, epochs=5, batch_size=128, verbose=0)
  kfold = StratifiedKFold(n_splits=4, shuffle=True)
  results = cross_val_score(estimator, X_input, Y_output, cv=kfold)
  print(labels[i], "baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
  stats.append(results.mean())

print("average: %.2f%% (%.2f%%)" % (np.mean(stats)*100, np.std(stats)*100))

anger baseline: 74.62% (1.95%)
anticipation baseline: 85.26% (0.03%)
disgust baseline: 73.05% (1.50%)
fear baseline: 81.40% (0.12%)
joy baseline: 79.38% (0.79%)
love baseline: 89.45% (0.00%)
optimism baseline: 76.21% (0.74%)
pessimism baseline: 88.02% (0.02%)
sadness baseline: 72.28% (0.43%)
surprise baseline: 94.56% (0.03%)
trust baseline: 94.62% (0.03%)
average: 82.62% (7.89%)


In [ ]:
#fit the model to one dimension
selection = 8
model = create_baseline(number_of_classes=1)
model.fit(X_train_embeddings, Y_train_clean[:,selection], epochs=20, validation_data=(X_dev_embeddings, Y_dev_clean[:,selection]))

Epoch 1/20
208/208 [==============================] - 3s 12ms/step - loss: 0.6121 - accuracy: 0.6981 - val_loss: 0.5700 - val_accuracy: 0.6984
Epoch 2/20
208/208 [==============================] - 2s 11ms/step - loss: 0.5605 - accuracy: 0.7062 - val_loss: 0.5251 - val_accuracy: 0.7362
Epoch 3/20
208/208 [==============================] - 2s 11ms/step - loss: 0.5115 - accuracy: 0.7483 - val_loss: 0.5014 - val_accuracy: 0.7534
Epoch 4/20
208/208 [==============================] - 2s 11ms/step - loss: 0.4864 - accuracy: 0.7639 - val_loss: 0.5051 - val_accuracy: 0.7557
Epoch 5/20
208/208 [==============================] - 2s 11ms/step - loss: 0.4801 - accuracy: 0.7708 - val_loss: 0.4835 - val_accuracy: 0.7615
Epoch 6/20
208/208 [==============================] - 2s 11ms/step - loss: 0.4567 - accuracy: 0.7791 - val_loss: 0.4948 - val_accuracy: 0.7557
Epoch 7/20
208/208 [==============================] - 2s 11ms/step - loss: 0.4500 - accuracy: 0.7951 - val_loss: 0.4886 - val_accuracy: 0.7580

In [ ]:
#fit the model to all dimensions
models = []
for i in range(class_no):
  model = create_baseline(number_of_classes=1)
  Y_train_split = Y_train_clean[:,i]
  Y_dev_split = Y_dev_clean[:,i]
  print("processing model for",labels[i])
  model.fit(X_train_embeddings, Y_train_split, epochs=10, validation_data=(X_dev_embeddings, Y_dev_split), verbose=0)
  models.append(model)

print("trained", len(models), "models")

processing model for anger
processing model for anticipation
processing model for disgust
processing model for fear
processing model for joy
processing model for love
processing model for optimism
processing model for pessimism
processing model for sadness
processing model for surprise
processing model for trust
trained 11 models


# Evaluate the classifiers

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results=[]
for i, model in enumerate(models):
  Y_dev_split = Y_dev_clean[:,i]
  partial = model.evaluate(X_dev_embeddings, Y_dev_split, batch_size=128, verbose=0)
  print(labels[i], "test loss, test acc:", partial)


Evaluate on test data
anger test loss, test acc: [0.4944448471069336, 0.7568807601928711]
anticipation test loss, test acc: [0.39562666416168213, 0.8589449524879456]
disgust test loss, test acc: [0.4880869388580322, 0.7568807601928711]
fear test loss, test acc: [0.25919538736343384, 0.9036697149276733]
joy test loss, test acc: [0.44426000118255615, 0.78899085521698]
love test loss, test acc: [0.29084280133247375, 0.8715596199035645]
optimism test loss, test acc: [0.4670793414115906, 0.7844036817550659]
pessimism test loss, test acc: [0.3221796154975891, 0.8910550475120544]
sadness test loss, test acc: [0.4782714545726776, 0.7683486342430115]
surprise test loss, test acc: [0.14201349020004272, 0.9598624110221863]
trust test loss, test acc: [0.18342220783233643, 0.9506880640983582]


In [ ]:
# Calculate F1 score for one dimension

true = Y_test[:,selection]
prediction = models[selection].predict(X_test_embeddings)

sample = 10

# Normalize output
from sklearn.preprocessing import Binarizer
transformer = Binarizer(threshold=0.5).fit(prediction)  # fit does nothing.
prediction = transformer.transform(prediction)
flat = [x[0] for x in prediction]

#print(labels[selection])
#print(X_test[0:5])
#print(true[0:5])
#print(flat[0:5])

recall = recall_score(y_true=true, y_pred=flat, average='weighted')
precision = precision_score(y_true=true, y_pred=flat, average='weighted')
score = f1_score(y_true=true, y_pred=flat, average='weighted')

print(recall, precision, score, "\n")

for i in range(sample):
  print(X_test[i])
  print("predicted",labels[selection], bool(flat[i]))
  print("expected",labels[selection], bool(true[i]), "\n")

0.7821417612764652 0.7736761494486198 0.775536220429787 

@Adnan__786__ @AsYouNotWish Dont worry Indian army is on its ways to dispatch all Terrorists to Hell
predicted sadness True
expected sadness False 

Academy of Sciences, eschews the normally sober tone of scientific papers and calls the massive loss of wildlife a “biological annihilation
predicted sadness False
expected sadness False 

I blew that opportunity -__- mad
predicted sadness False
expected sadness True 

This time in 2 weeks I will be 30... 😥
predicted sadness False
expected sadness True 

Deppression is real. Partners w/ depressed people truly dont understand the depth in which they affect us. Add in anxiety &amp;makes it worse
predicted sadness True
expected sadness True 

@POLITICOEurope Interesting choice of words... Are you confirming that governments fund terrorism? Bit of an open door, but still...
predicted sadness False
expected sadness False 

@ananavarro CNN should, for sure 😀
predicted sadness False
expect

In [ ]:
# Calculate F1 score for all dimensions

for i in range(class_no):
  true = Y_test[:,i]
  prediction = models[i].predict(X_test_embeddings)

  # Normalize output
  from sklearn.preprocessing import Binarizer
  transformer = Binarizer(threshold=0.5).fit(prediction)  # fit does nothing.
  prediction = transformer.transform(prediction)

  #calc
  from sklearn.metrics import f1_score, recall_score, precision_score

  recall = recall_score(y_true=true, y_pred=prediction, average='weighted')
  precision = precision_score(y_true=true, y_pred=prediction, average='weighted')
  score = f1_score(y_true=true, y_pred=prediction, average='weighted')

  print("%.2f %.2f %.2f" % (recall, precision, score), labels[i])

0.80 0.80 0.80 anger
0.87 0.81 0.81 anticipation
0.77 0.77 0.77 disgust
0.90 0.89 0.88 fear
0.82 0.82 0.82 joy
0.88 0.88 0.88 love
0.77 0.76 0.76 optimism
0.89 0.85 0.84 pessimism
0.78 0.77 0.78 sadness
0.95 0.90 0.92 surprise
0.95 0.91 0.93 trust


In [ ]:
# Test with popular tweets
obama = "No one is born hating another person because of the color of his skin or his background or his religion..."
culkin = '''Hey guys, wanna feel old?
I'm 40.
You're welcome.'''
mel = "IM OFFICIALLY CANCER FREE !!! 😭😭😭❤️❤️❤️"
kamala ="We did it, @JoeBiden."
obama2 = "Kobe was a legend on the court and just getting started in what would have been just as meaningful a second act. To lose Gianna is even more heartbreaking to us as parents. Michelle and I send love and prayers to Vanessa and the entire Bryant family on an unthinkable day."
trump = '''Tonight, 
@FLOTUS
 and I tested positive for COVID-19. We will begin our quarantine and recovery process immediately. We will get through this TOGETHER!'''
trump2 = '''@FoxNews
 daytime is not watchable. In a class with CNN & MSDNC. Check out 
@OANN
, 
@newsmax
 and others that are picking up the slack. Even a boring football game, kneeling and all, is better!'''
trump3 = '''...WOW, he just voted against me in a Big Court Decision on voter fraud (of which there was much!), despite many pages of dissent from three highly respected Justices. One thing has nothing to do with another, but we ended up losing 4-3 in a really incorrect ruling! Great...'''
abascal = '''No puedes trabajar.

No puedes salir por la noche.

No puedes cambiar de provincia.

No puedes elegir la educación de tu hijo.

No puedes saber cuánta gente ha muerto por la pandemia.

Pero, gracias al gobierno social delincuente, puedes ir al hospital a pedir que te maten.'''

loteria='''#LoteríaA3N | En la mítica 'gasolinera de la suerte' de Tenerife vuelve a ir todo sobre ruedas. Allí ha tocado #ElGordo y un quinto premio: "Este año todo han sido malas noticias y lo deseábamos mucho"'''

a3n="España podrá empezar a vacunar del #coronavirus este domingo tras el visto bueno de la Agencia Europea del Medicamento a la vacuna de Pfizer"
zoom="Zoom ha comunicado que eliminará el límite de 40 minutos en las llamadas entre usuarios gratuitos durante la Navidad"
paula="Con el círculo tan bonito tengo que hasta me sale Trump y va y me lo fastidia Pablo Casado...¡no me lo puedo creer!, jajaja."
rey='''Los del #mireyesfelipevi,
¿Si tanto os gusta por qué no os lo lleváis a vuestra casa y le pagáis el sueldo a él y a su familia?'''
abascal_reply='''Los delincuentes ladrones se protegen entre sí. Tu Abascal no puedes dar lecciones de moralidad, ni de  ética. Pues eso a defender a los de tu especie.. PARÁSITOS. Tu también tienes mucha experiencia e vivir del cuento. Limpiemos España de gandules tuuuu..'''
abascal_reply2='''Lo siento mucho Santi, pero el rey lleva el puto pin de la agenda globalista 2030, es uno de ellos... Su madre desde Bidelberg le mueve los hilos... Estáis solos en esto... Como 
@realDonaldTrump'''
disabled = '''Estando discapacitado mi ex pareja después de 10 años de relación me dejó y me culpó de mi salud. Tuve que cerrar mi empresa y tuve que irme a vivir a casa de mi padre viudo y mayor para que me cuidara. 
Todos tenemos la fuerza para seguir adelante, solo hay que encontrarla.'''

import re
regex = r"[.]{2,3}"
subst = "."
regexed = re.sub(regex, subst, disabled)

from nltk import tokenize
sentences = tokenize.sent_tokenize(regexed)
embeddings = laser.embed_sentences(sentences,lang='en')
print(sentences)
print(embeddings.shape)

from sklearn.preprocessing import Binarizer
threshold = 0.5
X=[[0]]
transformer = Binarizer(threshold=threshold).fit(X)  # fit does nothing.

for i, sentence in enumerate(sentences):
  print("\n", sentence)
  single_embedding = np.array([embeddings[i]])
  for j, model in enumerate(models):
    prediction = model.predict(single_embedding)
    # Normalize output    
    normalized = transformer.transform(prediction)
    result = bool(normalized[0,0])
    if result: print("guessed", labels[j], "with prob", prediction[0,0])

#for j, model in enumerate(models):
  #predictions = model.predict(embeddings)
  #for i, sentence in enumerate(sentences):
    #print(sentence)
    #print(predictions[i])
    #results = [x[0] for x in predictions[i]]
    #print("guessing",labels[j])
    #print(results)

['Estando discapacitado mi ex pareja después de 10 años de relación me dejó y me culpó de mi salud.', 'Tuve que cerrar mi empresa y tuve que irme a vivir a casa de mi padre viudo y mayor para que me cuidara.', 'Todos tenemos la fuerza para seguir adelante, solo hay que encontrarla.']
(3, 1024)

 Estando discapacitado mi ex pareja después de 10 años de relación me dejó y me culpó de mi salud.
guessed anger with prob 0.60721034
guessed sadness with prob 0.72548664

 Tuve que cerrar mi empresa y tuve que irme a vivir a casa de mi padre viudo y mayor para que me cuidara.

 Todos tenemos la fuerza para seguir adelante, solo hay que encontrarla.
guessed optimism with prob 0.7915448


In [ ]:
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

# Serialization
numpyData = {"array": embeddings}
encodedNumpyData = json.dumps(numpyData, cls=NumpyArrayEncoder)  # use dump() to write array into file
print("Printing JSON serialized NumPy array")
print(encodedNumpyData)

# Save to file
with open("/content/drive/MyDrive/Machine Learning/Datasets/SemEval-2018 Affect in Tweets E-c/embeddings.json", "w") as embeddings_file:
      embeddings_file.write(encodedNumpyData)

Printing JSON serialized NumPy array
{"array": [[0.024796972051262856, 0.045601099729537964, 0.002532974584028125, 0.004394921474158764, 0.010344580747187138, 0.004091605544090271, 0.012954209931194782, 0.005151262041181326, 0.010556516237556934, 0.03964399918913841, 0.01787501573562622, 0.01139589212834835, 0.013363514095544815, 0.013372454792261124, 0.001467935391701758, 0.039863284677267075, 0.03791657090187073, -0.00015601736959069967, 0.0019241657573729753, 0.00683590630069375, 0.019809693098068237, 0.02854272723197937, -4.746748527395539e-05, 0.039664123207330704, 0.02000419609248638, 0.02013053186237812, 0.029920151457190514, 0.009359641000628471, 0.0005113229271955788, 0.0012880692956969142, 5.5795877415221184e-05, 0.009435273706912994, 0.008825259283185005, 0.02114678919315338, 0.02514241263270378, 0.009719683788716793, 0.009697988629341125, 0.01720561273396015, -0.0019446113146841526, 0.001895297085866332, 0.007857268676161766, 0.007311737630516291, 0.00246702809818089, -3.13

In [ ]:
import re

regex = r"[.]{2,3}"
subst = "."

print(tokenize.sent_tokenize(re.sub(regex, subst, abascal_reply2)))

['Lo siento mucho Santi, pero el rey lleva el puto pin de la agenda globalista 2030, es uno de ellos.', 'Su madre desde Bidelberg le mueve los hilos.', 'Estáis solos en esto.', 'Como \n@realDonaldTrump']


# Save the models

In [ ]:
!pip install tensorflowjs
import tensorflowjs as tfjs

for i,model in enumerate(models):
  # serialize model to JSON
  model_json = model.to_json()
  with open("/content/drive/MyDrive/Machine Learning/Datasets/SemEval-2018 Affect in Tweets E-c/"+labels[i]+"-model.json", "w") as json_file:
      json_file.write(model_json)

  # serialize weights to HDF5
  model_weights = model.save_weights("/content/drive/MyDrive/Machine Learning/Datasets/SemEval-2018 Affect in Tweets E-c/"+labels[i]+"-weights.h5")
  
  # save as TensorFlow.js Layers format
  tfjs.converters.save_keras_model(model, "/content/drive/MyDrive/Machine Learning/Datasets/SemEval-2018 Affect in Tweets E-c/"+labels[i])

print("Saved models and weights to disk")

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


Saved models and weights to disk


In [ ]:
from nltk import download as nltkdownload
nltkdownload('punkt', download_dir='/content/drive/MyDrive/Machine Learning/nltk_data')

[nltk_data] Downloading package punkt to
[nltk_data]     /content/drive/MyDrive/Machine Learning/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install laserembeddings
!python -m laserembeddings download-models /content/drive/MyDrive/Laser


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!
